<h1>Data preparation<h1>

In [ ]:
import pandas as pd
import numpy as np

# Load dataset
data = pd.read_csv('path_to_your_dataset.csv')

# Assume 'Cluster' column exists after clustering the data
data['Cluster'] = ...  # Add your clustering logic or load the pre-computed clusters

# Rename columns for convenience
data = data.rename(columns={
    'RSRP (d Bm)-Dominant RSRP (d Bm)': 'RSRP_dBm',
    'Serving RS Info-Serving RSRP (d Bm)': 'Serving_RSRP_dBm',
    'Serving RS Info-Serving RS RSRQ (d B)': 'Serving_RSRQ_dB',
    'Serving RS Info-Serving RS SINR (d B)': 'Serving_SINR_dB'
})

# Drop rows with missing target values
data = data.dropna(subset=['RSRP_dBm', 'Serving_RSRP_dBm', 'Serving_RSRQ_dB', 'Serving_SINR_dB'])
data = data.fillna(0)  # Fill other missing values with 0 (or use other strategies)

# Define features and target variables
features = [
    'Latitude', 'Longitude', 'Serving Channel Info-DL EARFCN',
    'Serving Cell Info-Serving PCI', 'Serving RS Info-NR Best SS-RSRP',
    'Serving RS Info-NR Best SS-SINR', 'Data Throughput-RLC DL Throughput (kbps)',
    'Data Throughput-NR PDCP downlink throughput (Mbps)',
    '5G NR-NR Best SS-RSRP', '5G NR-NR PDCP downlink throughput (Mbps)',
    '5G NR-NR Best SS-SINR'
]

target_rsrp = 'RSRP_dBm'
target_rsrq = 'Serving_RSRQ_dB'
target_sinr = 'Serving_SINR_dB'

X = data[features]
y_rsrp = data[target_rsrp]
y_rsrq = data[target_rsrq]
y_sinr = data[target_sinr]
clusters = data['Cluster']


<h1>Graph construction<h1>

In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import StandardScaler

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create edge index based on clusters
edges = []
for cluster in data['Cluster'].unique():
    cluster_data = data[data['Cluster'] == cluster]
    indices = cluster_data.index.tolist()
    for i in indices:
        for j in indices:
            if i != j:
                edges.append([i, j])

edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

# Create node features and target tensors
x = torch.tensor(X_scaled, dtype=torch.float)
y_rsrp = torch.tensor(y_rsrp.values, dtype=torch.float)
y_rsrq = torch.tensor(y_rsrq.values, dtype=torch.float)
y_sinr = torch.tensor(y_sinr.values, dtype=torch.float)

# Create PyTorch Geometric Data object
data_rsrp = Data(x=x, edge_index=edge_index, y=y_rsrp)
data_rsrq = Data(x=x, edge_index=edge_index, y=y_rsrq)
data_sinr = Data(x=x, edge_index=edge_index, y=y_sinr)


<h1>GNN model training<h1>

In [ ]:
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool

class GNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, data.batch)
        x = self.fc(x)
        return x

# Initialize GNN models
gnn_rsrp = GNN(input_dim=X_scaled.shape[1], hidden_dim=64, output_dim=1)
gnn_rsrq = GNN(input_dim=X_scaled.shape[1], hidden_dim=64, output_dim=1)
gnn_sinr = GNN(input_dim=X_scaled.shape[1], hidden_dim=64, output_dim=1)

# Train the GNN models
def train_gnn(model, data, epochs=100, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = F.mse_loss(out, data.y)
        loss.backward()
        optimizer.step()
        if (epoch+1) % 10 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

# Train the models
train_gnn(gnn_rsrp, data_rsrp)
train_gnn(gnn_rsrq, data_rsrq)
train_gnn(gnn_sinr, data_sinr)


<h1>Identify Poor Coverage Areas<h1>

In [ ]:
gnn_rsrp.eval()
gnn_rsrq.eval()
gnn_sinr.eval()

rsrp_pred = gnn_rsrp(data_rsrp).detach().numpy()
rsrq_pred = gnn_rsrq(data_rsrq).detach().numpy()
sinr_pred = gnn_sinr(data_sinr).detach().numpy()

# Define thresholds for poor coverage
rsrp_threshold = -100  # Example threshold for RSRP
rsrq_threshold = -12  # Example threshold for RSRQ
sinr_threshold = 5  # Example threshold for SINR

poor_coverage = (rsrp_pred < rsrp_threshold) | (rsrq_pred < rsrq_threshold) | (sinr_pred < sinr_threshold)

# Combine results into a DataFrame for analysis
results = pd.DataFrame({
    'Latitude': X['Latitude'],
    'Longitude': X['Longitude'],
    'RSRP_Pred': rsrp_pred.flatten(),
    'RSRQ_Pred': rsrq_pred.flatten(),
    'SINR_Pred': sinr_pred.flatten(),
    'Poor_Coverage': poor_coverage.flatten()
})

# Filter poor coverage areas
poor_coverage_areas = results[results['Poor_Coverage']]
print(poor_coverage_areas)


<h1>Optimal Cell Placement<h1>

In [ ]:
from scipy.spatial import distance_matrix

# Calculate the centroid of poor coverage areas
centroid = poor_coverage_areas[['Latitude', 'Longitude']].mean().values

# Calculate distances from the centroid to all poor coverage areas
distances = distance_matrix([centroid], poor_coverage_areas[['Latitude', 'Longitude']].values)
poor_coverage_areas['Distance_to_Centroid'] = distances.flatten()

# Suggest optimal location for new cell tower (nearest to centroid)
optimal_location = poor_coverage_areas.loc[poor_coverage_areas['Distance_to_Centroid'].idxmin()]
print('Optimal Cell Tower Location:')
print(optimal_location[['Latitude', 'Longitude']])


<h1>Visualization<h1>

In [ ]:
import folium

# Create a map centered at the centroid
m = folium.Map(location=centroid, zoom_start=13)

# Add poor coverage areas to the map
for _, row in poor_coverage_areas.iterrows():
    folium.CircleMarker(location=[row['Latitude'], row['Longitude']],
                        radius=5, color='red', fill=True).add_to(m)

# Add optimal cell tower location to the map
folium.Marker(location=optimal_location[['Latitude', 'Longitude']],
              popup='Optimal Cell Tower Location', icon=folium.Icon(color='blue')).add_to(m)

# Save and display the map
m.save('network_coverage_map.html')
m
